In [3]:
#Generación Archivo
#importar librerias
import csv
import numpy as np
import pandas as pd
import random as rd
import string as st
import json
import os
import sys
import google.cloud.storage as storage
from datetime import datetime
from datetime import timedelta

In [4]:
#mount drive
# from google.colab import drive
# drive.mount('/content/drive')

# path = '/content/drive/Shareddrives/Infra_Grupo_Sarasa/TP_Final_Especificacion_1.xlsx'
path = './data/TP_Final_Especificacion_1.xlsx'
# path_destino = '/content/drive/Shareddrives/Infra_Grupo_Sarasa'
path_destino = './data'
# path_coord='/content/drive/Shareddrives/Infra_Grupo_Sarasa/ar.csv'
path_coord='./data/ar.csv'

In [8]:

# PATH = os.path.join(os.getcwd(),'/content/drive/Shareddrives/Infra_Grupo_Sarasa/usm-infra-grupo1-42c8c1c4178a.json' )
PATH = 'usm-infra-grupo1-42c8c1c4178a.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = PATH
storage_client = storage.Client()
# bucket_name = 'distribuidor_01'
#bucket = storage_client.create_bucket('distribuidor_01')

In [ ]:
# TO DO:
# chequear existencia bucket
# crear bucket (uno por distribuidor)(con carpetas ventas deuda etc etc)
# guardar data en buckets en vez de en las carpetas

# #upload file
# carpeta_destino =  path_destino + '/distribuidor_'+str(distribuidor)

# archivo_destino_v =  carpeta_destino +'/venta_'+fech_cierre_string+'.csv'
# archivo_destino_s =  carpeta_destino +'/stock_'+fech_cierre_string+'.csv'
# archivo_destino_m =  carpeta_destino + '/cliente_'+fech_cierre_string+'.csv'
# archivo_destino_d =  carpeta_destino + '/deuda_'+fech_cierre_string+'.csv'

# df_vta_cli.to_csv(archivo_destino_v, encoding='utf-8', index=False)
# df_stock.to_csv(archivo_destino_s, encoding='utf-8', index=False)
# df_clientes.to_csv(archivo_destino_m, encoding='utf-8', index=False)
# df_deuda.to_csv(archivo_destino_d, encoding='utf-8', index=False)
# secuencia = 0
# buket_name = 'var'+secuencia
# secuencia += 1



In [10]:
# Leer el archivo de especificaciones
df_sheets_desc = pd.read_excel(path, sheet_name=None)
df_cond_vta = df_sheets_desc['Condicion_Venta']
df_unidades = df_sheets_desc['Unidades']
df_tipo_negocio = df_sheets_desc['Tipo_Negocio']
df_cadena = df_sheets_desc['Cadena']
df_dias_visita = df_sheets_desc['Dias_Visita']
df_productos = df_sheets_desc['Productos']
df_cond_venta = df_sheets_desc['Condicion_Venta']
df_localidades = df_sheets_desc['Prov_Localidades']
df_estado = df_sheets_desc['Estado_cliente']
df_dias_visita = df_sheets_desc['Dias_Visita']

coordenadas = pd.read_csv(path_coord)

In [11]:
def generar_codigos_sku(productos):
  codigos_sku = {}
  for producto in productos:
    caracteres = st.ascii_uppercase + st.digits
    sku_codigo = ''.join(rd.choice(caracteres) for _ in range(10))
    codigos_sku[producto] = sku_codigo

  return codigos_sku

In [ ]:
#generar SKUs a partir de la lista de productos
codigos_sku = generar_codigos_sku(df_productos['nombre'])

#nombre de las columnas de los archivos cliente,venta,stock
columns_v = df_sheets_desc['Desc_Ventas']['Campo']
columns_s = df_sheets_desc['Desc_Stock']['Campo']
columns_m = df_sheets_desc['Desc_Clientes']['Campo']
columns_d = df_sheets_desc['Desc_Deuda']['Campo']

#cantidad de días desde la fecha actual hacia el pasado que generamos datos
cant_dias = 2

#fecha actual
fecha_actual = datetime.now()

#cantidad de distribuidores
cant_dist = 3

#cantidad de clientes
cant_clientes = 4

#cantidad sucursales
cant_sucursales = 5

In [21]:
#generar SKUs a partir de la lista de productos
codigos_sku = generar_codigos_sku(df_productos['nombre'])

#nombre de las columnas de los archivos cliente,venta,stock
columns_v = df_sheets_desc['Desc_Ventas']['Campo']
columns_s = df_sheets_desc['Desc_Stock']['Campo']
columns_m = df_sheets_desc['Desc_Clientes']['Campo']
columns_d = df_sheets_desc['Desc_Deuda']['Campo']


#cantidad de días desde la fecha actual hacia el pasado que generamos datos
cant_dias = 2

#fecha actual
fecha_actual = datetime.now()

#cantidad de distribuidores
cant_dist = 3

#cantidad de clientes
cant_clientes = 4

#cantidad sucursales
cant_sucursales = 5

for distribuidor in range(1,cant_dist):

  for d in range(0,cant_dias):

    #fecha de cierre
    fech_cierre = fecha_actual - timedelta(days=d)
    fech_cierre_string = fech_cierre.strftime('%Y-%m-%d')
    fech_cie_com_data = [ (fech_cierre - timedelta(hours=rd.randint(0, 23), minutes=rd.randint(0, 59), seconds=rd.randint(0, 59))).strftime('%Y-%m-%d %H:%M:%S') for _ in range(cant_clientes)]

    #generamos los datos de venta unidades
    vta_unidades_data = [(rd.randint(0,100)) for _ in range(cant_clientes)]

    #generamos los datos de venta en importes
    vta_importe_data = [round((rd.uniform(100,1000)),2) for _ in range(cant_clientes)]

    #generamos la condicion de venta
    cond_vta_data = [ rd.choice(df_cond_vta['codigo_condicion']) for _ in range(cant_clientes)]

    codigo_sucursal_base = (distribuidor * 100)  # Código base para el distribuidor

    #sucursales del distribuidor
    #el dist1 va a tener sucursales de la 26 a 50
    #el dist2 va a tener sucursales de la 51 a 75
    sucursales_dist = [(codigo_sucursal_base + rd.randint(10, 99)) for _ in range(cant_clientes)]

    #clientes de la sucursal del distribuidor
    clientes = [(codigo_sucursal_base + rd.randint(1000, 9999)) for _ in range(cant_clientes)]

    #Provincia y ciudades
    provincias = [(rd.choice(df_localidades.columns.values)) for _ in range(cant_clientes)]
    ciudades_aleatorias = []
    for _ in range(cant_clientes):
      for p in provincias:
          ciudades_provincia = df_localidades[p].dropna().tolist()
          ciudad_aleatoria = rd.choice(ciudades_provincia)
          ciudades_aleatorias.append(ciudad_aleatoria)

    #Estado
    estado = [(rd.choices(df_estado['nombre'], weights=df_estado['probabilidades'], k=1)[0]) for _ in range(cant_clientes)]

    #Nombre_ cliente
    nombre_cliente = [(f'Cliente_{_}') for _ in range(cant_clientes)]

    #CUIT
    cuit = [(rd.randint(10000000, 99999999) ) for _ in range(cant_clientes)]

    #Razon Social
    razon_social = [(f"Empresa_{_}") for _ in range(cant_clientes)]

    #Direccion
    direccion = [(f"Dirección_{_}") for _ in range(cant_clientes)]

    #Dias de visita
    dias_visita = [(rd.choice(df_dias_visita['codigo_dia'])) for _ in range(cant_clientes)]

    #Telefono
    telefono = [(f"{rd.randint(100, 999)}-{rd.randint(100, 999)}-{rd.randint(1000, 9999)}") for _ in range(cant_clientes)]

    #Fecha de alta y de baja
    fecha_alta = [(fech_cierre - timedelta(days=rd.randint(cant_dias,365))).strftime('%Y-%m-%d') for _ in range(cant_clientes)]
    fecha_baja = [(fech_cierre + timedelta(days=rd.randint(cant_dias,365))).strftime('%Y-%m-%d') if estado[_]=="I" else pd.NA for _ in range(cant_clientes)   ]

    #Coordenadas
    coordenada_latitud = coordenadas[ coordenadas['city'].isin(ciudades_aleatorias) ]['lat']    #agregar coordenada de acuerdo a la ciudad!!!!!!!!
    coordenada_longitud = coordenadas[ coordenadas['city'].isin(ciudades_aleatorias) ]['lng'] #agregar coordenada de acuerdo a la ciudad!!!!!!!!

    #Tipo de negocio
    tipo_negocio = [(rd.choice(df_tipo_negocio['nombre'])) for _ in range(cant_clientes)]

    #ARCHIVO STOCK
    #Unidad
    unidad = [(rd.choice(df_unidades['codigo_unidad'])) for _ in range(cant_clientes)]
    #Stock
    stock = [(rd.randint(100, 500) if unidad == "UNI" else rd.randint(1, 100)) for _ in range(cant_clientes)]
    #Producto
    producto=[]
    producto1=[]
    producto1 = list(codigos_sku.keys())
    producto1.extend(producto1)
    producto=producto1
    #SKU
    sku_codigo=[]
    sku_codigo1=[]
    sku_codigo1 = list(codigos_sku.values())
    sku_codigo1.extend(sku_codigo1)
    sku_codigo = sku_codigo1

    #DEUDA
    deuda_vencida = [(round(rd.uniform(0, 1000), 2)) for _ in range(cant_clientes)]
    deuda_total = [(round(rd.uniform(0, 10000), 2)) for _ in range(cant_clientes)]

    #data frame con data
    venta_clientes_data = list(zip(sucursales_dist,clientes,fech_cie_com_data,sku_codigo,vta_unidades_data,vta_importe_data,cond_vta_data))
    stock_data = list(zip(sucursales_dist, fech_cie_com_data, sku_codigo, producto, stock, unidad))
    clientes_data = list(zip(sucursales_dist, clientes, ciudades_aleatorias, provincias, estado, nombre_cliente, cuit, razon_social, direccion, dias_visita, telefono, fecha_alta, fecha_baja, coordenada_latitud, coordenada_longitud, cond_vta_data, deuda_vencida, tipo_negocio))
    deuda_data = list(zip(sucursales_dist,clientes, fech_cie_com_data,deuda_vencida,deuda_total ))

    df_vta_cli = pd.DataFrame(venta_clientes_data,columns=columns_v)
    df_stock = pd.DataFrame(stock_data,columns=columns_s)
    df_clientes = pd.DataFrame(clientes_data,columns=columns_m)
    df_deuda =  pd.DataFrame(deuda_data,columns=columns_d)

    #upload file
    
    

    #archivo_destino_v =  carpeta_destino +'/venta_'+fech_cierre_string+'.csv'
    #archivo_destino_s =  carpeta_destino +'/stock_'+fech_cierre_string+'.csv'
    #archivo_destino_m =  carpeta_destino + '/cliente_'+fech_cierre_string+'.csv'
    #archivo_destino_d =  carpeta_destino + '/deuda_'+fech_cierre_string+'.csv'

    #df_vta_cli.to_csv(archivo_destino_v, encoding='utf-8', index=False)
    #df_stock.to_csv(archivo_destino_s, encoding='utf-8', index=False)
    #df_clientes.to_csv(archivo_destino_m, encoding='utf-8', index=False)
    #df_deuda.to_csv(archivo_destino_d, encoding='utf-8', index=False)

            
  carpeta_destino = 'g1_distribuidor_' + str(distribuidor)
  bucket = storage_client.bucket(carpeta_destino)

  if not bucket.exists():
    bucket = storage_client.create_bucket(carpeta_destino)
  
  ###
  dataframesDict = {'venta': df_vta_cli, 'stock': df_stock, 'cliente': df_clientes, 'deuda': df_deuda}
  for key, dataframe in dataframesDict.items():
    archivo_destino = f'{key}_'+fech_cierre_string+'.csv'
    blob = bucket.blob(archivo_destino)
    blob.upload_from_string(dataframe.to_csv(), 'text/csv')
  ###

g1_distribuidor_1
g1_distribuidor_2


In [ ]:
print(timedelta(days=1))

1 day, 0:00:00


In [ ]:
def generate_sucursales_dist(codigo_sucursal_base):
"""
Generates a list of sucursales for the distributor.
"""
return [(codigo_sucursal_base + rd.randint(10, 99)) for _ in range(CANT_CLIENTES)]


def generate_clientes(codigo_sucursal_base):
"""
Generates a list of clients for the distributor.
"""
return [(codigo_sucursal_base + rd.randint(1000, 9999)) for _ in range(CANT_CLIENTES)]


def generate_provincias():
"""
Generates a list of provinces for the clients.
"""
return [(rd.choice(DF_LOCALIDADES.columns.values)) for _ in range(CANT_CLIENTES)]


def generate_ciudades_aleatorias(provincias):
"""
Generates a list of random cities for the clients based on their provinces.
"""
ciudades_aleatorias = []
for _ in range(CANT_CLIENTES):
    for p in provincias:
        ciudades_provincia = DF_LOCALIDADES[p].dropna().tolist()
        ciudad_aleatoria = rd.choice(ciudades_provincia)
        ciudades_aleatorias.append(ciudad_aleatoria)
return ciudades_aleatorias


def generate_estado():
"""
Generates a list of states for the clients.
"""
return [(rd.choices(DF_ESTADO['nombre'], weights=DF_ESTADO['probabilidades'], k=1)[0]) for _ in range(CANT_CLIENTES)]


def generate_nombre_cliente():
"""
Generates a list of client names.
"""
return [(f'Cliente_{_}') for _ in range(CANT_CLIENTES)]


def generate_cuit():
"""
Generates a list of CUIT numbers for the clients.
"""
return [(rd.randint(10000000, 99999999)) for _ in range(CANT_CLIENTES)]


def generate_razon_social():
"""
Generates a list of company names for the clients.
"""
return [(f"Empresa_{_}") for _ in range(CANT_CLIENTES)]


def generate_direccion():
"""
Generates a list of addresses for the clients.
"""
return [(f"Dirección_{_}") for _ in range(CANT_CLIENTES)]


def generate_dias_visita():
"""
Generates a list of visit days for the clients.
"""
return [(rd.choice(DF_DIAS_VISITA['codigo_dia'])) for _ in range(CANT_CLIENTES)]


def generate_telefono():
"""
Generates a list of phone numbers for the clients.
"""
return [(f"{rd.randint(100, 999)}-{rd.randint(100, 999)}-{rd.randint(1000, 9999)}") for _ in range(CANT_CLIENTES)]


def generate_fecha_alta(fech_cierre):
"""
Generates a list of registration dates for the clients.
"""
return [(fech_cierre - timedelta(days=rd.randint(CANT_DIAS, 365))).strftime('%Y-%m-%d') for _ in range(CANT_CLIENTES)]


def generate_fecha_baja(fech_cierre, estado):
"""
Generates a list of cancellation dates for the clients.
"""
return [(fech_cierre + timedelta(days=rd.randint(CANT_DIAS, 365))).strftime('%Y-%m-%d') if estado[_] == "I" else pd.NA for _ in range(CANT_CLIENTES)]


def generate_coordenadas(ciudades_aleatorias):
"""
Generates a list of coordinates for the clients based on their cities.
"""
coordenada_latitud = COORDENADAS[COORDENADAS['city'].isin(ciudades_aleatorias)]['lat']
coordenada_longitud = COORDENADAS[COORDENADAS['city'].isin(ciudades_aleatorias)]['lng']
return coordenada_latitud, coordenada_longitud


def generate_tipo_negocio():
"""
Generates a list of business types for the clients.
"""
return [(rd.choice(DF_TIPO_NEGOCIO['nombre'])) for _ in range(CANT_CLIENTES)]


def generate_unidad():
"""
Generates a list of units for the stock.
"""
return [(rd.choice(DF_UNIDADES['codigo_unidad'])) for _ in range(CANT_CLIENTES)]


def generate_stock(unidad):
"""
Generates a list of stock for the products.
"""
return [(rd.randint(100, 500) if unidad == "UNI" else rd.randint(1, 100)) for _ in range(CANT_CLIENTES)]


def generate_producto():
"""
Generates a list of products for the stock.
"""
producto = list(CODIGOS_SKU.keys())
producto.extend(producto)
return producto


def generate_sku_codigo():
"""
Generates a list of SKU codes for the stock.
"""
sku_codigo = list(CODIGOS_SKU.values())
sku_codigo.extend(sku_codigo)
import pandas as pd
import random as rd
from datetime import datetime, timedelta

CANT_CLIENTES = 100
COLUMNS_V = ['sucursal', 'cliente', 'fecha_cierre_compra', 'sku_codigo', 'vta_unidades', 'vta_importe', 'cond_vta']
COLUMNS_S = ['sucursal', 'fecha_cierre_compra', 'sku_codigo', 'producto', 'stock', 'unidad']
COLUMNS_M = ['sucursal', 'cliente', 'ciudad', 'provincia', 'estado', 'nombre_cliente', 'cuit', 'razon_social', 'direccion', 'dias_visita', 'telefono', 'fecha_alta', 'fecha_baja', 'coordenada_latitud', 'coordenada_longitud', 'cond_vta', 'deuda_vencida', 'tipo_negocio']
COLUMNS_D = ['sucursal', 'cliente', 'fecha_cierre_compra', 'deuda_vencida', 'deuda_total']


def generate_sucursales_dist(codigo_sucursal_base):
    """
    Generates a list of distributor branches.
    """
    return [codigo_sucursal_base + i for i in range(1, 6)]


def generate_clientes(codigo_sucursal_base):
    """
    Generates a list of clients for the distributor.
    """
    return [codigo_sucursal_base + i for i in range(1, CANT_CLIENTES + 1)]


def generate_provincias():
    """
    Generates a list of provinces in Argentina.
    """
    return ['Buenos Aires', 'CABA', 'Catamarca', 'Chaco', 'Chubut', 'Córdoba', 'Corrientes', 'Entre Ríos', 'Formosa', 'Jujuy', 'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén', 'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe', 'Santiago del Estero', 'Tierra del Fuego', 'Tucumán']


def generate_ciudades_aleatorias(provincias):
    """
    Generates a list of random cities in Argentina.
    """
    ciudades = []
    for provincia in provincias:
        ciudades.append(provincia + ' City')
    return ciudades


def generate_estado():
    """
    Generates a list of states for the clients.
    """
    return ['Activo' if rd.random() < 0.9 else 'Inactivo' for _ in range(CANT_CLIENTES)]


def generate_nombre_cliente():
    """
    Generates a list of client names.
    """
    return [f'Cliente {i}' for i in range(1, CANT_CLIENTES + 1)]


def generate_cuit():
    """
    Generates a list of client CUITs.
    """
    return [f'{rd.randint(20, 27)}-{rd.randint(10000000, 99999999)}-{rd.randint(0, 9)}' for _ in range(CANT_CLIENTES)]


def generate_razon_social():
    """
    Generates a list of client business names.
    """
    return [f'Empresa {i}' for i in range(1, CANT_CLIENTES + 1)]


def generate_direccion():
    """
    Generates a list of client addresses.
    """
    return [f'Calle {i} {rd.randint(1, 1000)}' for i in range(1, CANT_CLIENTES + 1)]


def generate_dias_visita():
    """
    Generates a list of days of the week for client visits.
    """
    return [rd.choice(['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']) for _ in range(CANT_CLIENTES)]


def generate_telefono():
    """
    Generates a list of client phone numbers.
    """
    return [f'11-{rd.randint(1000, 9999)}-{rd.randint(1000, 9999)}' for _ in range(CANT_CLIENTES)]


def generate_fecha_alta(fecha_actual):
    """
    Generates a list of client registration dates.
    """
    return [(fecha_actual - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(30)]


def generate_fecha_baja(fecha_actual, estado):
    """
    Generates a list of client cancellation dates.
    """
    return [fecha_actual.strftime('%Y-%m-%d') if estado[i] == 'Activo' else (fecha_actual - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(30)]


def generate_coordenadas(ciudades_aleatorias):
    """
    Generates a list of client coordinates.
    """
    coordenada_latitud = []
    coordenada_longitud = []
    for ciudad in ciudades_aleatorias:
        coordenada_latitud.append(rd.uniform(-90, 90))
        coordenada_longitud.append(rd.uniform(-180, 180))
    return coordenada_latitud, coordenada_longitud


def generate_tipo_negocio():
    """
    Generates a list of client business types.
    """
    return [rd.choice(['Minorista', 'Mayorista']) for _ in range(CANT_CLIENTES)]


def generate_unidad():
    """
    Generates a list of product units.
    """
    return [rd.choice(['Kg', 'L', 'U']) for _ in range(10)]


def generate_stock(unidad):
    """
    Generates a list of product stock for the clients.
    """
    return [rd.randint(0, 1000) for _ in range(10)] * 10


def generate_producto():
    """
    Generates a list of product names.
    """
    return [f'Producto {i}' for i in range(1, 101)]


def generate_sku_codigo():
    """
    Generates a list of product codes.
    """
    return [f'{rd.randint(100000, 999999)}-{rd.randint(10, 99)}' for _ in range(100)]


def generate_deuda_vencida():
    """
    Generates a list of overdue debts for the clients.
    """
    return [(round(rd.uniform(0, 1000), 2)) for _ in range(CANT_CLIENTES)]


def generate_deuda_total():
    """
    Generates a list of total debts for the clients.
    """
    return [(round(rd.uniform(0, 10000), 2)) for _ in range(CANT_CLIENTES)]


def generate_data():
    """
    Generates data for the distributor and clients.
    """
    data = []
    distribuidores = [51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
    fech_cie_com_data = [(datetime.now() - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(30)]
    vta_unidades_data = [rd.randint(1, 10) for _ in range(30)]
    vta_importe_data = [round(rd.uniform(100, 1000), 2) for _ in range(30)]
    cond_vta_data = ['Contado' if rd.random() < 0.8 else 'Cuenta Corriente' for _ in range(30)]

    for distribuidor in distribuidores:
        codigo_sucursal_base = distribuidor * 1000

        sucursales_dist = generate_sucursales_dist(codigo_sucursal_base)
        clientes = generate_clientes(codigo_sucursal_base)
        provincias = generate_provincias()
        ciudades_aleatorias = generate_ciudades_aleatorias(provincias)
        estado = generate_estado()
        nombre_cliente = generate_nombre_cliente()
        cuit = generate_cuit()
        razon_social = generate_razon_social()
        direccion = generate_direccion()
        dias_visita = generate_dias_visita()
        telefono = generate_telefono()
        fecha_alta = generate_fecha_alta(datetime.now())
        fecha_baja = generate_fecha_baja(datetime.now(), estado)
        coordenada_latitud, coordenada_longitud = generate_coordenadas(ciudades_aleatorias)
        tipo_negocio = generate_tipo_negocio()
        unidad = generate_unidad()
        stock = generate_stock(unidad)
        producto = generate_producto()
        sku_codigo = generate_sku_codigo()
        deuda_vencida = generate_deuda_vencida()
        deuda_total = generate_deuda_total()

        venta_clientes_data = list(zip(sucursales_dist, clientes, fech_cie_com_data, sku_codigo, vta_unidades_data, vta_importe_data, cond_vta_data))
        stock_data = list(zip(sucursales_dist, fech_cie_com_data, sku_codigo, producto, stock, unidad))
        clientes_data = list(zip(sucursales_dist, clientes, ciudades_aleatorias, provincias, estado, nombre_cliente, cuit, razon_social, direccion, dias_visita, telefono, fecha_alta, fecha_baja, coordenada_latitud, coordenada_longitud, cond_vta_data, deuda_vencida, tipo_negocio))
        deuda_data = list(zip(sucursales_dist, clientes, fech_cie_com_data, deuda_vencida, deuda_total))

        df_vta_cli = pd.DataFrame(venta_clientes_data, columns=COLUMNS_V)
        df_stock = pd.DataFrame(stock_data, columns=COLUMNS_S)
        df_clientes = pd.DataFrame(clientes_data, columns=COLUMNS_M)
        df_deuda = pd.DataFrame(deuda_data, columns=COLUMNS_D)

        data.append({
            'venta': df_vta_cli,
            'stock': df_stock,
            'cliente': df_clientes,
            'deuda': df_deuda,
            'distribuidor': distribuidor,
            'fecha_cierre': datetime.now().strftime('%Y-%m-%d')
        })

    return data


def generate_data_2():
    """
    Generates data for the distributor and clients.
    """
    data = []
    distribuidores = [51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
    fech_cie_com_data = [(datetime.now() - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(30)]
    vta_unidades_data = [rd.randint(1, 10) for _ in range(30)]
    vta_importe_data = [round(rd.uniform(100, 1000), 2) for _ in range(30)]
    cond_vta_data = ['Contado' if rd.random() < 0.8 else 'Cuenta Corriente' for _ in range(30)]

    for distribuidor in distribuidores:
        codigo_sucursal_base = distribuidor * 1000

        sucursales_dist = generate_sucursales_dist(codigo_sucursal_base)
        clientes = generate_clientes(codigo_sucursal_base)
        provincias = generate_provincias()
        ciudades_aleatorias = generate_ciudades_aleatorias(provincias)
        estado = generate_estado()
        nombre_cliente = generate_nombre_cliente()
        cuit = generate_cuit()
        razon_social = generate_razon_social()
        direccion = generate_direccion()
        dias_visita = generate_dias_visita()
        telefono = generate_telefono()
        fecha_alta = generate_fecha_alta(datetime.now())
        fecha_baja = generate_fecha_baja(datetime.now(), estado)
        coordenada_latitud, coordenada_longitud = generate_coordenadas(ciudades_aleatorias)
        tipo_negocio = generate_tipo_negocio()
        unidad = generate_unidad()
        stock = generate_stock(unidad)
        producto = generate_producto()
        sku_codigo = generate_sku_codigo()
        deuda_vencida = generate_deuda_vencida()
        deuda_total = generate_deuda_total()

        venta_clientes_data = list(zip(sucursales_dist, clientes, fech_cie_com_data, sku_codigo, vta_unidades_data, vta_importe_data, cond_vta_data))
        stock_data = list(zip(sucursales_dist, fech_cie_com_data, sku_codigo, producto, stock, unidad))
        clientes_data = list(zip(sucursales_dist, clientes, ciudades_aleatorias, provincias, estado, nombre_cliente, cuit, razon_social, direccion, dias_visita, telefono, fecha_alta, fecha_baja, coordenada_latitud, coordenada_longitud, cond_vta_data, deuda_vencida, tipo_negocio))
        deuda_data = list(zip(sucursales_dist, clientes, fech_cie_com_data, deuda_vencida, deuda_total))

        df_vta_cli = pd.DataFrame(venta_clientes_data, columns=COLUMNS_V)
        df_stock = pd.DataFrame(stock_data, columns=COLUMNS_S)
        df_clientes = pd.DataFrame(clientes_data, columns=COLUMNS_M)
        df_deuda = pd.DataFrame(deuda_data, columns=COLUMNS_D)

        data.append({
            'venta': df_vta_cli,
            'stock': df_stock,
            'cliente': df_clientes,
            'deuda': df_deuda,
            'distribuidor': distribuidor,
            'fecha_cierre': datetime.now().strftime('%Y-%m-%d')
        })

    return data


def upload_files(data, path_destino):
    """
    Uploads the generated files to the specified destination folder.
    """
    for d in data:
        distribuidor = d['distribuidor']
        fecha_cierre_string = d['fecha_cierre']
        carpeta_destino = f"{path_destino}/distribuidor_{distribuidor}"
        archivo_destino_v = f"{carpeta_destino}/venta_{fecha_cierre_string}.csv"
        archivo_destino_s = f"{carpeta_destino}/stock_{fecha_cierre_string}.csv"
        archivo_destino_m = f"{carpeta_destino}/cliente_{fecha_cierre_string}.csv"
        archivo_destino_d = f"{carpeta_destino}/deuda_{fecha_cierre_string}.csv"
        d['venta'].to_csv(archivo_destino_v, encoding='utf-8', index=False)
        d['stock'].to_csv(archivo_destino_s, encoding='utf-8', index=False)
        d['cliente'].to_csv(archivo_destino_m, encoding='utf-8', index=False)
        d['deuda'].to_csv(archivo_destino_d, encoding='utf-8', index=False)


def main(path_destino):
    """
    Main function to generate and upload the files.
    """
    data = generate_data()
    upload_files(data, path_destino)